<a href="https://colab.research.google.com/github/midnightBlueNebula/text-classification-spam-detector/blob/main/fcc_sms_text_classification_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [103]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras import layers
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import re
import string

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.10.0-dev20220616


In [104]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-06-16 19:04:20--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2022-06-16 19:04:20 (8.35 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2022-06-16 19:04:20--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[====

In [115]:
train_data = pd.read_csv(train_file_path, sep="\t", names=['type', 'message'])
test_data = pd.read_csv(test_file_path, sep="\t", names=['type', 'message'])

print(train_data)
print(test_data)

      type                                            message
0      ham  ahhhh...just woken up!had a bad dream about u ...
1      ham                           you can never do nothing
2      ham  now u sound like manky scouse boy steve,like! ...
3      ham  mum say we wan to go then go... then she can s...
4      ham  never y lei... i v lazy... got wat? dat day ü ...
...    ...                                                ...
4174   ham  just woke up. yeesh its late. but i didn't fal...
4175   ham  what do u reckon as need 2 arrange transport i...
4176  spam  free entry into our £250 weekly competition ju...
4177  spam  -pls stop bootydelious (32/f) is inviting you ...
4178   ham  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]
      type                                            message
0      ham  i am in hospital da. . i will return home in e...
1      ham         not much, just some textin'. how bout you?
2      ham  i probably won't eat at all today

In [116]:
train_data["type"] = train_data["type"].map({"ham": 0, "spam": 1})
test_data["type"] = test_data["type"].map({"ham": 0, "spam": 1})

train_labels = train_data["type"]
test_labels = test_data["type"]

train_data = train_data["message"].values.flatten()
test_data = test_data["message"].values.flatten()

In [162]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')


vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=300
)

vectorize_layer.adapt(train_data)

In [176]:
model = keras.Sequential()

model.add(vectorize_layer)
model.add(layers.Embedding(10000, 10))
model.add(layers.Dropout(0.2))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1))

In [177]:
model.compile(loss=keras.losses.binary_crossentropy, 
              optimizer=tf.optimizers.Adam(learning_rate=0.001),
              metrics="accuracy")

In [182]:
model.fit(train_data, train_labels, validation_split=0.2, epochs=20, verbose=1)

Epoch 1/20
105/105 [==============================] - 2s 20ms/step - loss: 0.0339 - accuracy: 0.9925 - val_loss: 0.0164 - val_accuracy: 0.9928
Epoch 2/20
105/105 [==============================] - 1s 12ms/step - loss: 0.0295 - accuracy: 0.9925 - val_loss: 0.0157 - val_accuracy: 0.9940
Epoch 3/20
105/105 [==============================] - 2s 15ms/step - loss: 0.0755 - accuracy: 0.9829 - val_loss: 0.1649 - val_accuracy: 0.9689
Epoch 4/20
105/105 [==============================] - 2s 15ms/step - loss: 0.0411 - accuracy: 0.9910 - val_loss: 0.0159 - val_accuracy: 0.9952
Epoch 5/20
105/105 [==============================] - 1s 14ms/step - loss: 0.0304 - accuracy: 0.9937 - val_loss: 0.0155 - val_accuracy: 0.9952
Epoch 6/20
105/105 [==============================] - 1s 13ms/step - loss: 0.0246 - accuracy: 0.9940 - val_loss: 0.0153 - val_accuracy: 0.9952
Epoch 7/20
105/105 [==============================] - 1s 14ms/step - loss: 0.0271 - accuracy: 0.9949 - val_loss: 0.0153 - val_accuracy: 0.9952

In [183]:
def return_class_name(key):
  return {0: "ham", 1: "spam"}[key]

In [184]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  result = model.predict([pred_text])[0][0]
  print(result)
  prediction = [result, return_class_name(round(result))]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 41ms/step
-0.09827843
[-0.09827843, 'ham']


In [185]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 39ms/step
-0.09827843
1/1 [==============================] - 0s 41ms/step
0.55891687
1/1 [==============================] - 0s 40ms/step
-0.119169936
1/1 [==============================] - 0s 39ms/step
1.1946967
1/1 [==============================] - 0s 42ms/step
1.353781
1/1 [==============================] - 0s 38ms/step
-0.11598234
1/1 [==============================] - 0s 47ms/step
-0.1110674
You passed the challenge. Great job!
